<a href="https://colab.research.google.com/github/abhi147p/Abhishek_INFO5731_Spring2023/blob/main/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon.

(2) Collect the top 10000 User Reviews of a film recently in 2023 or 2022 (you can choose any film) from IMDB.

(3) Collect all the reviews of the top 1000 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from [Semantic Scholar](https://www.semanticscholar.org).

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using a hashtag (you can use any hashtag) from Twitter. 


In [212]:
# Write your code here

import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

def get_soup(url):
    """
        params1: url (contains the url of google scholar page)
        return: soup (fetching the url page data and then further converted to html parser)
    """
  # headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}
    try:
        # requesting for data using requests with url and headers for authentication
        data = requests.get(url)
        # print(f"Extracted the data with response as {data.status_code}")
        if data.status_code != 200:
            raise Exception("Failed to fetch data")
    except Exception as ex:
        print(f"Exception occurred as {data.text} with status_code {data.status_code}")
        return None
    soup = BeautifulSoup(data.text, "html.parser")
    return soup


final_reviews = []

def get_review_data(soup):
  final_reviews = []
  try:
    # soup = get_soup(url)
    # print(soup.prettify())
    # reviews_list = soup.findAll("div", attrs={"class": "lister-item mode-detail imdb-user-review  collapsable"})
    # print(reviews_list[0].prettify())
    # imdb_reviews = [data.find("div", attrs={"class": "text show-more__control"}).content for data in reviews_list]
    imdb_reviews = [list_item for list_item in soup.find_all("div", attrs={"class": "review-container"})]
    final_reviews = [review.find("div", attrs={"class": "text show-more__control"}).text for review in imdb_reviews]
  except Exception as ex:
    print(ex)
  
  return final_reviews

def web_scrape_imdb():
  url = "https://www.imdb.com/title/tt14614892/reviews?ref_=tturv_ql_10000"
  print(url)
  soup=get_soup(url)
  final_reviews=get_review_data(soup)

  load_more = soup.select(".load-more-data")
  flag = False
  url = "https://www.imdb.com/"
  if len(load_more):
      ajaxurl = load_more[0]['data-ajaxurl']
      url = 'https://www.imdb.com/' + ajaxurl + "?ref_=undefined&paginationKey="
      try:
          key = load_more[0]['data-key']
          flag = True
      except KeyError:
          pass

  while flag:
      url_new = url + key
      soup = get_soup(url_new)
      reviews_new = get_review_data(soup)
      final_reviews.extend(reviews_new)
      load_more = soup.select(".load-more-data")
      if len(load_more):
          key = load_more[0]['data-key']
      else:
          flag = False
  
  return final_reviews

final_reviews = web_scrape_imdb()
print(len(final_reviews))
for x in final_reviews:
  print(x)

https://www.imdb.com/title/tt14614892/reviews?ref_=tturv_ql_10000
105
The entire movie looked like Video game art from Budokai Tenkaichi cut scenes. The art style was completely different than the original Dragon Ball Super series. It was incredibly disappointing and definitely ruined any enjoyment I had watching the movie.Gohans transformation was incredibly Anticlimactic and boring, and absolutely no back story, he just randomly transforms from being far weaker than he was in the Tournament of Power... Again, the animation of the characters was so off it looked like fan art from photoshop.Lastly is the creation of Cell Max. This was one of the worst parts of the movie, being that they claimed he was far superior to the original Cell. But he could not talk, he didn't heal automatically, or regenerate, and he was unrealistically large like a giant. No back story to how they improved him from the original (did they get cells from other aliens that visit the planet like Beerus, or Whis e

In [213]:
import pandas as pd

df = pd.DataFrame(final_reviews, columns=["Text"])
df.to_csv("DragonBallMovie_Reviews.csv")
print(df.shape)
df.head()

(105, 1)


,Text
0,The entire movie looked like Video game art fr...
1,"I'm not a Dragon Ball fan, the only times I've..."
2,"I've seen the movie on cinema, and me laugh so..."
3,"To be honest, at first I was really concerned ..."
4,As A Dragon Ball Fan From India It Was Delight...


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [214]:
# Write your code here
import nltk, re, string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
nltk.download('stopwords')
nltk.download("wordnet")
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def tokenize_sentence(s):
  reviews = []
  for x in s:
    reviews.append(nltk.word_tokenize(x))
  return reviews

def remove_puntutation_and_special_chars(s):
  """
    params1: string containing sentence
    return: string with no punctuations
  """
  # punctuation = """!()-[]{};+|=-_~`:'"\,<>./?@#$%^&*_~"""
  # s1 = s1.replace(punctuation, "")
  return "".join([x for x in s if x not in string.punctuation])

def remove_nums(s):
  return re.sub(r'[0-9]', '', s)

def remove_stopwords(s):
  stop = stopwords.words("english")
  return " ".join(x for x in s.split() if x not in stop)

def lower_all(s):
  return " ".join(x.lower() for x in s.split())

def stemming(s):
  st = PorterStemmer()
  # stems = []
  # for x in s:
  #   stems.append(" ".join([st.stem(word) for word in s.split()]))
  # return " ".join([st.stem(word) for word in s])
  return [[st.stem(term) for term in doc] for doc in s]

def lemmatize(s):
  lemmatizer = WordNetLemmatizer()
  # return " ".join([Word(word).lemmatize() for word in s.split()])
  return [[lemmatizer.lemmatize(term) for term in doc] for doc in s]

df["text_modified"] = df["Text"].replace("\n","")
operations = [remove_puntutation_and_special_chars, remove_nums, lower_all, remove_stopwords]
for op in operations:
  df["text_modified"] = df["text_modified"].apply(op)
print(df.shape)
df.to_csv("After cleaning data.csv")
df.head()

(105, 2)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Text,text_modified
0,The entire movie looked like Video game art fr...,entire movie looked like video game art budoka...
1,"I'm not a Dragon Ball fan, the only times I've...",im dragon ball fan times ive watched back chil...
2,"I've seen the movie on cinema, and me laugh so...",ive seen movie cinema laugh much like humor mo...
3,"To be honest, at first I was really concerned ...",honest first really concerned cgianimation sin...
4,As A Dragon Ball Fan From India It Was Delight...,dragon ball fan india delight watch theatre ma...


In [215]:
words = lemmatize(stemming(tokenize_sentence(df["text_modified"])))
words

[['entir',
  'movi',
  'look',
  'like',
  'video',
  'game',
  'art',
  'budokai',
  'tenkaichi',
  'cut',
  'scene',
  'art',
  'style',
  'complet',
  'differ',
  'origin',
  'dragon',
  'ball',
  'super',
  'seri',
  'incred',
  'disappoint',
  'definit',
  'ruin',
  'enjoy',
  'watch',
  'moviegohan',
  'transform',
  'incred',
  'anticlimact',
  'bore',
  'absolut',
  'back',
  'stori',
  'randomli',
  'transform',
  'far',
  'weaker',
  'tournament',
  'power',
  'anim',
  'charact',
  'look',
  'like',
  'fan',
  'art',
  'photoshoplastli',
  'creation',
  'cell',
  'max',
  'one',
  'worst',
  'part',
  'movi',
  'claim',
  'far',
  'superior',
  'origin',
  'cell',
  'could',
  'talk',
  'didnt',
  'heal',
  'automat',
  'regener',
  'unrealist',
  'larg',
  'like',
  'giant',
  'back',
  'stori',
  'improv',
  'origin',
  'get',
  'cell',
  'alien',
  'visit',
  'planet',
  'like',
  'beeru',
  'whi',
  'etc',
  'definit',
  'kept',
  'closer',
  'origin',
  'perfect',
  'ce

# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [229]:
# Write your code here
# 1) POS TAGGING
nltk.download('averaged_perceptron_tagger')
import spacy
nlp = spacy.load('en_core_web_sm')

# POS tagging
# Noun: NN, NNS
# Verb: VB, VBD, VBG, VBN, VBP, VBZ
# Adjective: JJ, JJR, JJS
# Adverb: RB, RBR, RBS
def tags_count(text, tag, print_val=False, model=nlp):
    '''This function returns the number of nouns in an item'''
    doc = model(text)
    # Generate list of POS tags
    pos = [token.pos_ for token in doc]
    name_pos = [(token, token.pos_) for token in doc]
    if print_val:
      print(name_pos)
    # Return number of nouns
    return pos.count(tag)

# def pos_tag(words_set):
#   return [nltk.pos_tag(words) for words in words_set]

# words_pos = pos_tag(words)
# print(words_pos)
# pos_count = {}
# for i in range(len(words_pos)):
#   tag = words_pos[i][1]
#   # print(tag[1])
#   pos_count[tag] = pos_count.get(tag, 0) + 1

# pos_count
# ===================================================
text_list = list(df["text_modified"])
noun, verb, adj, adv = 0, 0, 0, 0
# to print tags
for text in text_list:
  tags_count(text, "NOUN", True, nlp)
for text in text_list:
  noun += tags_count(text, "NOUN", model=nlp)
  verb += tags_count(text, "VERB", model=nlp)
  adj += tags_count(text, "ADJ", model=nlp)
  adv += tags_count(text, "ADV", model=nlp)

# noun, verb, adj, adv
print(f"No of nouns: {noun}")
print(f"No of verbs: {verb}")
print(f"No of adjectives: {adj}")
print(f"No of adverbs: {adv}")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[(entire, 'ADJ'), (movie, 'NOUN'), (looked, 'VERB'), (like, 'SCONJ'), (video, 'NOUN'), (game, 'NOUN'), (art, 'NOUN'), (budokai, 'NOUN'), (tenkaichi, 'PROPN'), (cut, 'VERB'), (scenes, 'NOUN'), (art, 'NOUN'), (style, 'NOUN'), (completely, 'ADV'), (different, 'ADJ'), (original, 'ADJ'), (dragon, 'NOUN'), (ball, 'PROPN'), (super, 'PROPN'), (series, 'PROPN'), (incredibly, 'ADV'), (disappointing, 'ADJ'), (definitely, 'ADV'), (ruined, 'VERB'), (enjoyment, 'NOUN'), (watching, 'VERB'), (moviegohans, 'PROPN'), (transformation, 'NOUN'), (incredibly, 'ADV'), (anticlimactic, 'ADJ'), (boring, 'ADJ'), (absolutely, 'ADV'), (back, 'ADV'), (story, 'NOUN'), (randomly, 'ADV'), (transforms, 'VERB'), (far, 'ADV'), (weaker, 'ADJ'), (tournament, 'NOUN'), (power, 'NOUN'), (animation, 'NOUN'), (characters, 'NOUN'), (looked, 'VERB'), (like, 'SCONJ'), (fan, 'NOUN'), (art, 'NOUN'), (photoshoplastly, 'ADV'), (creation, 'NOUN'), (cell, 'NOUN'), (max, 'PROPN'), (one, 'NUM'), (worst, 'ADJ'), (parts, 'NOUN'), (movie, 'N

In [222]:
# 3) EXTRACT ALL ENTITIES AND THEIR COUNT
text_list = list(df["text_modified"])

count_entity = {}
person_names, organizations, locations, product_names, date = [], [], [], [], []
entities = []
for text in text_list:
  doc = nlp(text)
  
  for ent in doc.ents:
      # print(ent.text, ent.start_char, ent.end_char, ent.label_)
      """
        ent.text: entity_name
        ent.start_char: start index of entity name
        ent.end_char: end index of entity name
        ent.label_: category of entity
      """
      entities.append([ent.text, ent.label_])
      if ent.label_ == "PERSON":
        person_names.append(ent.text)
      elif ent.label_ == "ORG":
        organizations.append(ent.text)
      elif ent.label_ == "GPE":
        locations.append(ent.text)
      elif ent.label_ == "PRODUCT":
        product_names.append(ent.text)
      elif ent.label_ == "DATE":
        date.append(ent.text)
      print(ent.label_, ent.text)
      count_entity[ent.label_] = count_entity.get(ent.label_, 0) + 1

count_entity

PERSON max
ORG dragon ball fan
ORG moviesi
ORDINAL first
PERSON max
ORG dragon ball fan
GPE india
DATE many years
CARDINAL one
PERSON gohan blanco
ORDINAL first
PERSON kakaroto goku
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
ORDINAL first
ORDINAL second
ORG loyal
DATE years
PERSON max
DATE year
PERSON max
PERSON max
GPE canada
CARDINAL zero
LOC north america
GPE canada
PERSON franchisegoku vegeta step
DATE last summer
PERSON vin diesel
TIME minutes
PERSON max
ORG dragon ball fan
ORG moviesi
ORDINAL first
PERSON max
ORG dragon ball fan
GPE india
DATE many years
CARDINAL one
PERSON gohan blanco
ORDINAL first
PERSON kakaroto goku
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON kakaroto
PERSON k

{'PERSON': 182,
 'ORG': 28,
 'ORDINAL': 28,
 'GPE': 21,
 'DATE': 28,
 'CARDINAL': 14,
 'LOC': 7,
 'TIME': 7}

In [218]:
# 2) print out the constituency parsing trees and dependency parsing trees

# dependency parsing tree
import spacy

text_list = list(df["text_modified"])
dep_data = []
for text in text_list:
  doc = nlp(text)
  # print the dependency parse tree

  for token in doc:
      dep_data.append([token.text, token.dep_, token.head.text, token.head.pos_,
              [child for child in token.children]])

dep_df = pd.DataFrame(dep_data, columns=["text", "dependency", "head_text", "head_pos", "children"])
print(df.shape)
dep_df.head()

(105, 2)


,text,dependency,head_text,head_pos,children
0,entire,amod,movie,NOUN,[]
1,movie,nsubj,looked,VERB,[entire]
2,looked,ROOT,looked,VERB,"[movie, cut, ruined, transforms, whis, etc, ke..."
3,like,mark,cut,VERB,[]
4,video,compound,game,NOUN,[]


In [219]:
# Constituency parse tree

import spacy

text_list = list(df["text_modified"])
dep_data = []
for text in text_list:
  doc = nlp(text)
  # print the dependency parse tree

  for chunk in doc.noun_chunks:
      dep_data.append((chunk.text, chunk.root.dep_, 
            chunk.root.head.text, chunk.root.text))

dep_df = pd.DataFrame(dep_data, columns=["text", "dependency", "head_text", "root"])
print(df.shape)
dep_df.head()

(105, 2)


,text,dependency,head_text,root
0,entire movie,nsubj,looked,movie
1,video game art budokai tenkaichi,nsubj,cut,tenkaichi
2,scenes art style,dobj,cut,style
3,completely different original dragon ball supe...,nsubj,ruined,series
4,enjoyment,dobj,ruined,enjoyment


In [220]:
# Visualizing the parse trees
from spacy import displacy
doc = nlp(list(df["text_modified"])[0])
# Since this is an interactive Jupyter environment, we can use displacy.render here
displacy.render(doc, style='dep', jupyter=True)

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

**Constituency parsing**, is process of identifying structure of sentence such as different phrases of grammer(noun, adjective etc). It follows top-down approach. 


**Dependency parsing** is process of identifying grammatical relationships between words. It follows bottom-up approach. 